In [23]:
import sys
import pandas as pd
import smtplib, ssl
import mimetypes
from getpass import getpass
from email.mime.multipart import MIMEMultipart
from email import encoders
from email.message import Message
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from email.mime.image import MIMEImage
from email.mime.text import MIMEText

In [24]:
drinks = pd.read_csv('http://bit.ly/drinksbycountry')
drinks.head()

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
0,Afghanistan,0,0,0,0.0,Asia
1,Albania,89,132,54,4.9,Europe
2,Algeria,25,0,14,0.7,Africa
3,Andorra,245,138,312,12.4,Europe
4,Angola,217,57,45,5.9,Africa


In [25]:
# Approach #1 on deleting column:
del drinks['continent']

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol
0,Afghanistan,0,0,0,0.0
1,Albania,89,132,54,4.9
2,Algeria,25,0,14,0.7
3,Andorra,245,138,312,12.4
4,Angola,217,57,45,5.9
...,...,...,...,...,...
188,Venezuela,333,100,3,7.7
189,Vietnam,111,2,1,2.0
190,Yemen,6,0,0,0.1
191,Zambia,32,19,4,2.5


In [26]:
# Approach #2 on deleting column:
drinks.drop(columns=['total_litres_of_pure_alcohol'], inplace=True)

,country,beer_servings,spirit_servings,wine_servings
0,Afghanistan,0,0,0
1,Albania,89,132,54
2,Algeria,25,0,14
3,Andorra,245,138,312
4,Angola,217,57,45
...,...,...,...,...
188,Venezuela,333,100,3
189,Vietnam,111,2,1
190,Yemen,6,0,0
191,Zambia,32,19,4


In [27]:
# save the transformed dataframe to a new csv file
fileToSend = "attachment.csv"
csv_data = drinks.to_csv(fileToSend, index=False)

In [28]:
emailfrom = "app.tester.031490@gmail.com"
emailto = "chan.dev.031490@gmail.com"
username = "app.tester.031490@gmail.com"
port = 465  # For SSL
password = getpass("Type your password and press enter: ")

context = ssl.create_default_context()

In [29]:
# Setup message
msg = MIMEMultipart()
msg["From"] = emailfrom
msg["To"] = emailto
msg["Subject"] = "This is an email from a python script"
msg.preamble = "Great"

content = f"""
This is a sample email content
"""

msg.attach(MIMEText(content, "plain"))

In [30]:
ctype, encoding = mimetypes.guess_type(fileToSend)
if ctype is None or encoding is not None:
    ctype = "application/octet-stream"

print(ctype)
print(encoding)


text/csv
None


In [31]:
maintype, subtype = ctype.split("/", 1)

if maintype == "text":
    fp = open(fileToSend)
    # Note: we should handle calculating the charset
    attachment = MIMEText(fp.read(), _subtype=subtype)
    fp.close()
elif maintype == "image":
    fp = open(fileToSend, "rb")
    attachment = MIMEImage(fp.read(), _subtype=subtype)
    fp.close()
elif maintype == "audio":
    fp = open(fileToSend, "rb")
    attachment = MIMEAudio(fp.read(), _subtype=subtype)
    fp.close()
else:
    fp = open(fileToSend, "rb")
    # should we replace it with MIMEApplication?
    attachment = MIMEBase(maintype, subtype)
    attachment.set_payload(fp.read())
    fp.close()
    encoders.encode_base64(attachment)
attachment.add_header("Content-Disposition", "attachment", filename=fileToSend)
msg.attach(attachment)

In [32]:
with smtplib.SMTP_SSL("smtp.gmail.com", port, context=context) as server:
    server.login(username,password)
    server.sendmail(emailfrom, emailto, msg.as_string())
    print('Email sent')

Email sent
